In [1]:
import numpy as np
import tensorflow as tf
from environment import Environment
from actor import Actor

2023-11-25 19:36:35.696526: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-25 19:36:35.859655: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 19:36:35.859719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 19:36:35.890753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 19:36:35.957545: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-25 19:36:35.959031: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [72]:
n_samples = 2
n_locations = 3
max_demand = 5
max_capacity = 20
env = Environment(n_samples, n_locations, max_demand, max_capacity)
act = Actor(n_locations)
optimizer = tf.keras.optimizers.Adam()

with tf.GradientTape(persistent=True) as tape:
    range_idx = tf.expand_dims(tf.range(n_samples, dtype=tf.float32), 1)
    range_idx = tf.constant([[0], [1], [2], [3], [4.0]])
    tape.watch(range_idx)

    actions = None
    for i in range(6):
        logits = act(env, training=True)
        logits = logits - env.mask * 100000

        logits2 = tf.nn.softmax(logits * 100)
        
        idx = tf.argmax(logits, 1)
        abc = env.locations * tf.tile(tf.expand_dims(logits2, -1), [1, 1, 2])
        action = tf.reduce_sum(abc, axis=1)

        state = env.update(idx)
        
        actions.append(action)
 
    
    acts = tf.convert_to_tensor(actions)
    acts_2 = tf.concat((tf.expand_dims(actions[-1], 0), actions[:-1]), 0)
    
    distances = tf.math.sqrt(tf.reduce_sum(tf.math.square(acts_2 - acts), -1) + 1e-12)
    all_path = tf.reduce_sum(distances, axis=0)
    
    # dist = tf.math.sqrt(tf.reduce_sum((tf.math.square(x1 - x2)), 1)) * 100000
    # R = reward_func(actions)

    R = tf.reduce_mean(all_path * 10000000 )

In [86]:
acts.shape

TensorShape([6, 2, 2])

In [80]:
tf.expand_dims(actions[0], 1)

<tf.Tensor: shape=(2, 1, 2), dtype=float32, numpy=
array([[[ 0.9129071 ,  0.7854059 ]],

       [[-0.07478051, -0.48097593]]], dtype=float32)>

In [76]:
actions[0]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 0.9129071 ,  0.7854059 ],
       [-0.07478051, -0.48097593]], dtype=float32)>

In [73]:
tape.gradient(R, act.trainable_variables)

[<tf.Tensor: shape=(3, 64), dtype=float32, numpy=
 array([[     0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,      0.      ,
              0.      ,      0.      ,      0.      ,

In [5]:
tape.gradient(tf.math.sqrt(R + 2), act.trainable_variables)

[None, None, None, None, None, None, None, None]

In [74]:
distances

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[1.8840435e+00, 5.0049597e-01],
       [1.8840435e+00, 5.0049597e-01],
       [1.4816787e+00, 8.8295609e-01],
       [1.4816787e+00, 8.8295609e-01],
       [1.0000000e-06, 1.0000000e-06],
       [1.0000000e-06, 1.0000000e-06]], dtype=float32)>

In [47]:
grads

[<tf.Tensor: shape=(3, 64), dtype=float32, numpy=
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [92]:
distances

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[1.6007552, 0.630413 ],
       [1.3515015, 0.7179877],
       [0.6792868, 1.194415 ],
       [0.       , 0.       ],
       [0.       , 0.       ],
       [0.       , 0.       ]], dtype=float32)>

In [443]:
acts.shape

TensorShape([20, 5, 2])

In [438]:
act_2.shape

TensorShape([20, 5, 2])

In [433]:
actions[-2]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.5547297 ,  0.33316517],
       [-0.9946034 ,  0.68414736],
       [-0.8453646 , -0.35012817],
       [-0.05941892, -0.5306032 ],
       [ 0.10520387, -0.8080895 ]], dtype=float32)>

In [412]:
grads[3][0]

<tf.Tensor: shape=(), dtype=float32, numpy=0.033893973>

In [401]:
act.trainable_variables[2][0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([-0.0789131 , -0.11420071, -0.07258695,  0.14461856,  0.20010315,
        0.21251209, -0.02950175, -0.16454795,  0.04056363,  0.17194687,
       -0.05777016, -0.00283304, -0.1989951 ,  0.08465316,  0.02717163,
       -0.20818457,  0.13845952, -0.09363864, -0.08414758,  0.11545043,
       -0.00914209, -0.2144667 ,  0.17920916,  0.12533903,  0.08253516,
        0.07560847, -0.12364417, -0.05225706,  0.07257701, -0.03708631,
        0.01309452, -0.17924425, -0.05799545,  0.11297812, -0.14737372,
       -0.04073825, -0.06614798,  0.10706209, -0.04454208,  0.14828064,
        0.17970586,  0.1935883 , -0.06552164, -0.00096052,  0.02632249,
       -0.04328397, -0.14863361, -0.01338717, -0.08311972, -0.17754105,
        0.06155334,  0.15120105, -0.16108026,  0.05747295, -0.11872796,
        0.18264787,  0.11795197,  0.06367145, -0.07073121,  0.14580195,
        0.07202268,  0.11720204, -0.00423585, -0.19099985], dtype=float32)>

In [402]:
grads_and_vars = zip(grads, act.trainable_variables)
optimizer.apply_gradients(grads_and_vars)

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>

In [403]:
act.trainable_variables[2][0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([-0.07809148, -0.11350412, -0.07316923,  0.14512576,  0.20082732,
        0.2130993 , -0.03015529, -0.16389719,  0.04125327,  0.1724223 ,
       -0.05871786, -0.00356625, -0.1984957 ,  0.08390189,  0.02645704,
       -0.20718668,  0.13914937, -0.09297912, -0.0831503 ,  0.11606952,
       -0.00832639, -0.21384464,  0.17990837,  0.12433819,  0.08285898,
        0.07496566, -0.12285034, -0.05287823,  0.07265899, -0.03644953,
        0.01233923, -0.17826451, -0.05762524,  0.11229046, -0.14834021,
       -0.04145354, -0.06699543,  0.10636219, -0.04364842,  0.14888765,
        0.1791678 ,  0.19299397, -0.06625613, -0.00038339,  0.02552432,
       -0.04254162, -0.14913069, -0.01384566, -0.08381084, -0.17698313,
        0.06093655,  0.15055577, -0.1612429 ,  0.05822559, -0.11935053,
        0.18330917,  0.11718134,  0.06300978, -0.07015739,  0.14677118,
        0.07200583,  0.11817075, -0.00379111, -0.19173887], dtype=float32)>

In [378]:
logits2[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([7.8159737e-30, 8.4592861e-35, 2.8145103e-15, 3.6260216e-03,
       0.0000000e+00, 6.1474402e-20, 9.6628922e-01, 0.0000000e+00,
       3.0084765e-02, 1.8081004e-13], dtype=float32)>

In [243]:
grads

[<tf.Tensor: shape=(10, 64), dtype=float32, numpy=
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.000

<tf.Tensor: shape=(5, 10, 2), dtype=float32, numpy=
array([[[-1.9736651 , -1.7213359 ],
        [-0.80834454,  0.26693672],
        [-0.654081  ,  1.7988795 ],
        [ 0.09345616, -0.46745378],
        [-0.12102263, -0.02670967],
        [ 3.574896  , -2.13068   ],
        [-0.6051694 , -0.6587835 ],
        [ 0.30703264, -0.32688642],
        [-1.0769366 ,  0.02709234],
        [-0.7743957 , -0.00848668]],

       [[ 0.2880399 ,  0.31462753],
        [-0.33387375, -0.01580391],
        [ 0.20033199,  0.21663345],
        [-0.6820592 , -0.41886666],
        [ 0.34131414,  0.07970672],
        [-1.2044654 , -0.7863991 ],
        [ 0.31076115,  0.14374001],
        [ 0.65338176,  0.5110428 ],
        [-2.4521687 ,  2.3524606 ],
        [-0.26951563, -0.17772862]],

       [[-0.74040276,  4.419273  ],
        [ 2.991199  , -0.6235747 ],
        [-3.7646883 , -4.9126115 ],
        [ 1.784937  ,  1.0757633 ],
        [ 2.9011421 , -0.6256206 ],
        [ 0.93070304, -0.10184076],
        

In [222]:
tf.reshape(logits, (5, 1, 10)).shape

TensorShape([5, 1, 10])

In [213]:
logits.shape

TensorShape([5, 10])

In [206]:
indices = tf.constant(tf.cast(tf.expand_dims(idx2, -1), dtype=tf.int32))
updates = tf.ones(shape=5, dtype=tf.float32)
scatter = tf.scatter_nd(indices, updates, [5, 10])
scatter

InvalidArgumentError: {{function_node __wrapped__ScatterNd_device_/job:localhost/replica:0/task:0/device:CPU:0}} Dimensions [1,2) of input[shape=[5,10]] must match dimensions [1,1) of updates[shape=[5]] [Op:ScatterNd]

In [207]:
indices

<tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[8],
       [8],
       [8],
       [0],
       [0]], dtype=int32)>

In [210]:
indices2

<tf.Tensor: shape=(4, 1), dtype=int32, numpy=
array([[4],
       [3],
       [1],
       [7]], dtype=int32)>

In [209]:
indices2 = tf.constant([[4], [3], [1], [7]])
updates2 = tf.constant([9, 10, 11, 12.0])
shape2 = tf.constant([8])
scatter2 = tf.scatter_nd(indices2, updates2, shape2)

In [203]:
indices

<tf.Tensor: shape=(4, 1), dtype=int32, numpy=
array([[4],
       [3],
       [1],
       [7]], dtype=int32)>

In [194]:
indices

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[8.],
       [8.],
       [8.],
       [0.],
       [0.]], dtype=float32)>

In [ ]:
tf.scatter_nd()

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([8., 8., 8., 0., 0.], dtype=float32)>

In [187]:
grads

[<tf.Tensor: shape=(10, 64), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [28]:
idx = np.array([0, 2, 1])

BatchSequence = tf.expand_dims(tf.cast(tf.range(3), tf.int64), 1)

batched_idx = tf.concat([BatchSequence, idx.reshape(-1, 1)], 1)

In [393]:
with tf.GradientTape(persistent=True) as gradient_tape:
    res = act(env)
    res2 = act(env)
    argmax = tf.argmax(res, 1)
    argmax2 = tf.argmax(res2, 1)

    range_idx = tf.expand_dims(tf.range(n_samples, dtype=tf.float32), 1)
    argmax_idx = tf.cast(tf.concat([range_idx, tf.expand_dims(tf.cast(argmax, tf.float32), -1)], 1), dtype=tf.int32)

    argmax_idx2 = tf.cast(tf.concat([range_idx, tf.expand_dims(tf.cast(argmax2, tf.float32), -1)], 1), dtype=tf.int32)

    # old_locations_idx = tf.cast(
    #     tf.concat(
    #         [range_idx, np.array([0] * n_samples).reshape(n_samples, 1)], 1
    #     ),
    #     dtype=tf.int32
    # )
    # old_locations = tf.zeros(shape=new_locations.shape) # tf.gather_nd(env.locations, old_locations_idx)
    new_locations = tf.gather_nd(env.locations, argmax_idx)
    old_locations = tf.gather_nd(env.locations, argmax_idx2)

    loss = tf.reduce_sum(new_locations - old_locations)

gradients = gradient_tape.gradient(tf.constant(loss), act.trainable_variables)
grads_and_vars = zip(gradients, act.trainable_variables)
optimizer.apply_gradients(grads_and_vars)

ValueError: No gradients provided for any variable: (['actor_2/dense_8/kernel:0', 'actor_2/dense_8/bias:0', 'actor_2/dense_9/kernel:0', 'actor_2/dense_9/bias:0', 'actor_2/dense_10/kernel:0', 'actor_2/dense_10/bias:0', 'actor_2/dense_11/kernel:0', 'actor_2/dense_11/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'actor_2/dense_8/kernel:0' shape=(10, 64) dtype=float32, numpy=
array([[ 1.60265326e-01,  1.01168007e-01,  1.31321818e-01,
        -2.10709125e-01, -1.95295185e-01,  2.17770904e-01,
         1.09577447e-01, -1.44582689e-02,  1.45250022e-01,
         1.07636690e-01, -2.26784050e-01,  3.26275527e-02,
        -1.03380114e-01,  2.50906378e-01,  1.77229851e-01,
        -1.77407578e-01,  2.44342357e-01,  4.06339765e-02,
         2.44153172e-01,  7.10038841e-02, -1.00888520e-01,
        -1.17701113e-01,  1.97643548e-01, -6.61384165e-02,
         2.53881544e-01, -8.26842040e-02, -2.78604597e-01,
         2.32795447e-01,  2.70272881e-01,  1.43675476e-01,
         8.66126120

argmax2

In [ ]:
idx = tf.expand_dims(tf.argmax(prob, 1), 1)

In [392]:
argmax2

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([4, 6, 4, 6, 6])>

In [388]:
new_locations

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.18297458, -0.32414532],
       [ 0.07010579,  0.6692085 ],
       [-0.68093157,  0.5190487 ],
       [ 0.43853927, -0.91252923],
       [ 0.291641  ,  0.9541609 ]], dtype=float32)>

In [386]:
-0.04366803 - -0.18297458

0.13930655

In [384]:
old_locations

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.04366803, -0.6686702 ],
       [ 0.6036005 ,  0.18465686],
       [-0.65384126,  0.84083104],
       [-0.6377001 ,  0.26406264],
       [-0.5929303 ,  0.22703886]], dtype=float32)>

In [385]:
new_locations

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.18297458, -0.32414532],
       [ 0.07010579,  0.6692085 ],
       [-0.68093157,  0.5190487 ],
       [ 0.43853927, -0.91252923],
       [ 0.291641  ,  0.9541609 ]], dtype=float32)>

In [376]:
env.locations[1, 6]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.07010579, 0.6692085 ], dtype=float32)>

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[-0.18297458, -0.32414532],
       [ 0.07010579,  0.6692085 ],
       [-0.68093157,  0.5190487 ],
       [ 0.43853927, -0.91252923],
       [ 0.291641  ,  0.9541609 ]], dtype=float32)>

In [369]:
env.locations[1, 2]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.21057177,  0.07490659], dtype=float32)>

In [367]:
[[1, 2]]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.21057177,  0.07490659], dtype=float32)>

In [366]:
argmax

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([4, 6, 4, 6, 6])>